In [15]:
import torch.nn as nn
import torch
import numpy as np
#linear regression is simplest nueral network

In [16]:
#Input (temp,rainfall,humidity)
inputs=np.array([[73,67,43],[91,88,64],[87,134,58],
                [102,43,37],[69,96,70],[73,67,43],
                [91,88,64],[87,134,58],[102,43,37],
                [69,96,70],[73,67,43],[91,88,64],
                [87,134,58],[102,43,37],[69,96,70]],dtype="float32")

In [17]:
#Targets (apples,oranges)
targets=np.array([[56,70],[81,101],[119,133],
                 [22,37],[103,119],[56,70],
                 [81,101],[119,133],[22,37],
                 [103,119],[56,70],[81,101],
                 [119,133],[22,37],[103,119]],dtype="float32")

In [18]:
inputs=torch.from_numpy(inputs) #convert to tensors
targets=torch.from_numpy(targets)

In [22]:
from torch.utils.data import TensorDataset
#Tensordataset allows access to rows from inputs and targets as tuples and provides
# standard ApI for working with many different types of datasets in Pytorch

In [23]:
train_ds=TensorDataset(inputs,targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [24]:
from torch.utils.data import DataLoader

In [28]:
#Define data  loader
#data Loader can split the data into batches of a predefined size while training
#It also provides other utilities like shuffling and random sampling of the data
batch_size=5
train_dl=DataLoader(train_ds,batch_size,shuffle=True)
#Dataloader(pytorch_dataset,batchsize)
#shuffle tells the loader before creating batches shuffle the roes in the data

In [32]:
for xb,yb in train_dl:
    print("Batch")
    print(xb)
    print(yb)

Batch
tensor([[102.,  43.,  37.],
        [102.,  43.,  37.],
        [ 73.,  67.,  43.],
        [ 87., 134.,  58.],
        [ 69.,  96.,  70.]])
tensor([[ 22.,  37.],
        [ 22.,  37.],
        [ 56.,  70.],
        [119., 133.],
        [103., 119.]])
Batch
tensor([[102.,  43.,  37.],
        [ 91.,  88.,  64.],
        [ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 69.,  96.,  70.]])
tensor([[ 22.,  37.],
        [ 81., 101.],
        [ 56.,  70.],
        [ 81., 101.],
        [103., 119.]])
Batch
tensor([[ 87., 134.,  58.],
        [ 73.,  67.,  43.],
        [ 69.,  96.,  70.],
        [ 87., 134.,  58.],
        [ 91.,  88.,  64.]])
tensor([[119., 133.],
        [ 56.,  70.],
        [103., 119.],
        [119., 133.],
        [ 81., 101.]])


In [33]:
#shuffling helps randomize the algorithm which reduces the chances of loss

In [37]:
#Define Model
#nn.Linear(number of inputs(temperatur,rainfall,humidity),number of outputs(yield of apples,yield of orange))
model=nn.Linear(3,2) 
print(model.weight)
print(model.bias)     #automatically created randomly initialized weights and biases

Parameter containing:
tensor([[-0.4292, -0.3111, -0.3537],
        [ 0.3612,  0.1618,  0.5084]], requires_grad=True)
Parameter containing:
tensor([ 0.4101, -0.2528], requires_grad=True)


In [39]:
list(model.parameters())  #every model has a .parameters object which contains a list where all the paameter matrices are there 

[Parameter containing:
 tensor([[-0.4292, -0.3111, -0.3537],
         [ 0.3612,  0.1618,  0.5084]], requires_grad=True),
 Parameter containing:
 tensor([ 0.4101, -0.2528], requires_grad=True)]

In [40]:
preds=model(inputs)
preds

tensor([[-66.9732,  58.8230],
        [-88.6595,  79.4008],
        [-99.1291,  82.3498],
        [-69.8323,  62.3642],
        [-83.8279,  75.7989],
        [-66.9732,  58.8230],
        [-88.6595,  79.4008],
        [-99.1291,  82.3498],
        [-69.8323,  62.3642],
        [-83.8279,  75.7989],
        [-66.9732,  58.8230],
        [-88.6595,  79.4008],
        [-99.1291,  82.3498],
        [-69.8323,  62.3642],
        [-83.8279,  75.7989]], grad_fn=<AddmmBackward>)

In [59]:
import torch.nn.functional as F

In [60]:
#the nn.functional package contains many useful loss functions and several other utilities

In [61]:
#Define loss function
loss_fn=F.mse_loss  #mean squre error


In [62]:
loss=loss_fn(model(inputs),targets) 
print(loss)


tensor(14049.1465, grad_fn=<MseLossBackward>)


In [63]:
#model.parameters() is passed to optiim.SGD where SGD stands for stochastic gradient descent.
#It is called stochastic because samples are selected in batches often with random shuffling instead as a single group

In [64]:
#Define optomizer
opt=torch.optim.SGD(model.parameters(),lr=1e-5)
#model.paraters() is passed so that optimizer knoaws whic matrices should be modified during the update step
#lr is the leraning rate which controls the amount by which the parameters are modified


In [67]:
#utility function to train the model
def fit(num_epochs,model,loss_fn,opt,train_dl):
    
    #Reapeat for given number of epochs
    for epoch in range(num_epochs):
        
        #train with bathches of data
        for xb,yb in train_dl:
            
            #Generate predictions
            pred=model(xb)
            
            #Calculate loss
            loss=loss_fn(pred,yb)
            
            #compute gradients
            #model.parameters passed to optimizer so by loss.backward() the w.grad aand b.grad will be updated in the optimizer
            loss.backward() 
            
            #update parameters using gradients
            #in this step it multiplies the gradient with the learning rate and subtracts it from the weight and bais
            opt.step()
            
            #reset the gradients to zero
            opt.zero_grad()
            
        #print the progress
        if (epoch+1)%10==0:
            print("Epoch [{}/{}], Loss: {:.4f}".format(epoch+1,num_epochs,loss))
            

In [68]:
fit(100,model,loss_fn,opt,train_dl)

Epoch [10/100], Loss: 577.8735
Epoch [20/100], Loss: 317.5182
Epoch [30/100], Loss: 99.8747
Epoch [40/100], Loss: 136.2946
Epoch [50/100], Loss: 65.3473
Epoch [60/100], Loss: 62.9182
Epoch [70/100], Loss: 86.6332
Epoch [80/100], Loss: 20.4843
Epoch [90/100], Loss: 16.0766
Epoch [100/100], Loss: 20.5934


In [69]:
preds=model(inputs)
preds


tensor([[ 58.5239,  71.5392],
        [ 80.2256, 100.0877],
        [121.1907, 132.4174],
        [ 28.1444,  44.1819],
        [ 94.4212, 113.8837],
        [ 58.5239,  71.5392],
        [ 80.2256, 100.0877],
        [121.1907, 132.4174],
        [ 28.1444,  44.1819],
        [ 94.4212, 113.8837],
        [ 58.5239,  71.5392],
        [ 80.2256, 100.0877],
        [121.1907, 132.4174],
        [ 28.1444,  44.1819],
        [ 94.4212, 113.8837]], grad_fn=<AddmmBackward>)

In [70]:
#compare with targets
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])